# DCGANs

**Deep Convolutional Generative Adversarial Networks **

## 1. Background

## 2. Definition

The Difference between the Simple GAN and the DCGAN
The generator of the simple GAN is a simple fully connected network. The generator of the DCGAN uses the transposed convolution technique to perform up-sampling of 2D image size.
FYI — the transposed convolution is also known as:
Fractionally-strided convolution
Deconvolution


## 3. Training